In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import manifold
from sklearn.manifold import TSNE
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.ticker import NullFormatter
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt

from time import time

In [2]:
df_train = pd.read_csv('train_genelevel_90%.tsv', sep='\t',index_col=0)
#df.set_index(df["Unnamed: 0"], inplace=True)
df_train.shape

(55169, 407)

In [3]:
df_train

,205_2,251_1,474_3,178_4,313_1,484_4,288_1,106_4,215_2,184_3,...,330_2,11_4,480_1,314_1,371_3,443_2,311_1,225_2,150_4,289_2
ENSG00000000419,202.0,243.00,230.0,270.0,232.00,231.00,174.00,164.00,244.0,207.0,...,257.00,274.00,230.00,327.00,252.0,285.0,237.00,223.0,278.00,159.0
ENSG00000000457,261.0,369.00,355.0,375.0,317.00,273.00,254.00,198.00,229.0,344.0,...,335.00,297.00,348.00,389.00,461.0,272.0,419.00,376.0,312.00,380.0
ENSG00000000460,194.0,188.00,234.0,236.0,385.00,154.00,144.00,253.00,218.0,214.0,...,331.00,198.00,192.00,274.00,251.0,322.0,364.00,129.0,322.00,252.0
ENSG00000000938,25.0,15.00,14.0,14.0,14.00,14.00,10.00,15.00,10.0,20.0,...,20.00,18.00,43.00,85.00,13.0,8.0,17.00,14.0,23.00,2.0
ENSG00000000971,428.0,91.06,83.0,184.0,167.78,138.79,105.76,196.97,236.0,61.0,...,118.77,163.48,200.57,266.55,175.0,344.0,146.84,67.0,367.51,97.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283693,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,...,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0
ENSG00000283695,0.0,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,0.0,...,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0
ENSG00000283696,5.0,1.00,1.0,5.0,1.00,1.00,0.00,2.00,3.0,3.0,...,3.00,0.00,2.00,4.00,1.0,2.0,5.00,0.0,1.00,4.0
ENSG00000283698,1.0,1.00,0.0,0.0,0.00,0.00,0.00,0.00,0.0,1.0,...,0.00,0.00,1.00,0.00,0.0,0.0,0.00,1.0,0.00,0.0


# Normalize across the column (by patient)

In [4]:
## get the sum for the column
sum_col = df_train.sum(axis=0)
sum_col

205_2    11339776.43
251_1    11474174.19
474_3    11493209.25
178_4    13767712.58
313_1    10777905.28
            ...     
443_2    11284345.75
311_1    11414900.39
225_2    10615704.62
150_4    10083242.19
289_2     8009566.15
Length: 407, dtype: float64

In [5]:
## sum across the column, then divide by each library depth
new_df = df_train.div(df_train.sum(axis=0), axis=1)
new_df

,205_2,251_1,474_3,178_4,313_1,484_4,288_1,106_4,215_2,184_3,...,330_2,11_4,480_1,314_1,371_3,443_2,311_1,225_2,150_4,289_2
ENSG00000000419,1.781340e-05,2.117799e-05,2.001182e-05,1.961110e-05,2.152552e-05,2.148744e-05,0.000022,1.835732e-05,2.528430e-05,1.616092e-05,...,1.662916e-05,0.000028,2.295356e-05,2.679032e-05,1.904284e-05,2.525623e-05,2.076234e-05,2.100661e-05,2.757050e-05,1.985126e-05
ENSG00000000457,2.301633e-05,3.215918e-05,3.088780e-05,2.723764e-05,2.941202e-05,2.539425e-05,0.000032,2.216310e-05,2.372994e-05,2.685679e-05,...,2.167614e-05,0.000031,3.472973e-05,3.186983e-05,3.483630e-05,2.410419e-05,3.670641e-05,3.541922e-05,3.094243e-05,4.744327e-05
ENSG00000000460,1.710792e-05,1.638462e-05,2.035985e-05,1.714155e-05,3.572123e-05,1.432496e-05,0.000018,2.831952e-05,2.259007e-05,1.670742e-05,...,2.141732e-05,0.000020,1.916123e-05,2.244816e-05,1.896727e-05,2.853511e-05,3.188815e-05,1.215181e-05,3.193417e-05,3.146238e-05
ENSG00000000938,2.204629e-06,1.307284e-06,1.218111e-06,1.016872e-06,1.298954e-06,1.302269e-06,0.000001,1.679023e-06,1.036242e-06,1.561441e-06,...,1.294098e-06,0.000002,4.291318e-06,6.963845e-06,9.823685e-07,7.089467e-07,1.489282e-06,1.318801e-06,2.281012e-06,2.497014e-07
ENSG00000000971,3.774325e-05,7.936083e-06,7.221656e-06,1.336460e-05,1.556703e-05,1.291014e-05,0.000013,2.204781e-05,2.445531e-05,4.762396e-06,...,7.685000e-06,0.000017,2.001650e-05,2.183780e-05,1.322419e-05,3.048471e-05,1.286389e-05,6.311404e-06,3.644760e-05,1.211052e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283693,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
ENSG00000283695,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
ENSG00000283696,4.409258e-07,8.715224e-08,8.700790e-08,3.631685e-07,9.278241e-08,9.301923e-08,0.000000,2.238697e-07,3.108726e-07,2.342162e-07,...,1.941147e-07,0.000000,1.995962e-07,3.277103e-07,7.556681e-08,1.772367e-07,4.380240e-07,0.000000e+00,9.917445e-08,4.994028e-07
ENSG00000283698,8.818516e-08,8.715224e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,7.807206e-08,...,0.000000e+00,0.000000,9.979808e-08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.420006e-08,0.000000e+00,0.000000e+00


In [6]:
## get the maximum of library depth
max_ld = df_train.sum(axis=0).max()
max_ld

34678509.490000084

In [7]:
## multiply by the largest library depth value
new_df2 = new_df.multiply(max_ld)
new_df2

,205_2,251_1,474_3,178_4,313_1,484_4,288_1,106_4,215_2,184_3,...,330_2,11_4,480_1,314_1,371_3,443_2,311_1,225_2,150_4,289_2
ENSG00000000419,617.742242,734.421290,693.979985,680.083747,746.472899,745.152502,757.493166,636.604390,876.821887,560.436479,...,576.674369,982.341697,795.995208,929.048373,660.377140,875.848314,720.006874,728.478033,956.103747,688.412194
ENSG00000000457,798.171907,1115.232329,1071.143020,944.560760,1019.965125,880.634775,1105.765886,768.583349,822.918902,931.353377,...,751.696162,1064.801036,1204.375359,1105.198217,1208.070880,835.897339,1272.923546,1228.285831,1073.037299,1645.261848
ENSG00000000460,593.277203,568.194249,706.049202,594.443572,1238.758906,496.768335,626.890896,982.078724,783.390047,579.388438,...,742.720685,709.867358,664.482956,778.468667,657.756596,989.554938,1105.833343,421.406575,1107.429520,1091.068383
ENSG00000000938,76.453248,45.334648,42.242260,35.263602,45.045778,45.160758,43.534090,58.226011,35.935323,54.148452,...,44.877383,64.533396,148.816495,241.495754,34.067075,24.585216,51.646063,45.734047,79.102109,8.659273
ENSG00000000971,1308.879602,275.211533,250.436255,463.464480,539.841479,447.704397,460.416536,764.585163,848.073628,165.152779,...,266.504338,586.106645,694.142430,757.302275,458.595236,1057.164281,446.100462,218.870082,1263.948518,419.974735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283693,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000283695,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000283696,15.290650,3.022310,3.017304,12.594143,3.217556,3.225768,0.000000,7.763468,10.780597,8.122268,...,6.731607,0.000000,6.921697,11.364506,2.620544,6.146304,15.190018,0.000000,3.439222,17.318546
ENSG00000283698,3.058130,3.022310,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.707423,...,0.000000,0.000000,3.460849,0.000000,0.000000,0.000000,0.000000,3.266718,0.000000,0.000000


# Low Variance filtering

In [8]:
new_df2.var(axis=1)

ENSG00000000419    27373.390944
ENSG00000000457    62571.649147
ENSG00000000460    47053.298734
ENSG00000000938      822.965476
ENSG00000000971    80528.134653
                       ...     
ENSG00000283693        0.020778
ENSG00000283695        0.000000
ENSG00000283696       34.597071
ENSG00000283698        1.885344
ENSG00000283699        0.000000
Length: 55169, dtype: float64

In [9]:
## get the minimum across the row
min_val = new_df2.min(axis=1)
min_val

ENSG00000000419    379.655439
ENSG00000000457    298.118700
ENSG00000000460    264.994400
ENSG00000000938      2.742792
ENSG00000000971     64.829175
                      ...    
ENSG00000283693      0.000000
ENSG00000283695      0.000000
ENSG00000283696      0.000000
ENSG00000283698      0.000000
ENSG00000283699      0.000000
Length: 55169, dtype: float64

In [10]:
## get the maximum across the row
max_val = new_df2.max(axis=1)
max_val

ENSG00000000419    1436.393185
ENSG00000000457    1892.789062
ENSG00000000460    1773.461180
ENSG00000000938     241.495754
ENSG00000000971    2254.352527
                      ...     
ENSG00000283693       2.285361
ENSG00000283695       0.000000
ENSG00000283696      36.321853
ENSG00000283698       8.254363
ENSG00000283699       0.000000
Length: 55169, dtype: float64

In [11]:
max_min_diff = max_val - min_val
max_min_diff

ENSG00000000419    1056.737746
ENSG00000000457    1594.670362
ENSG00000000460    1508.466780
ENSG00000000938     238.752963
ENSG00000000971    2189.523353
                      ...     
ENSG00000283693       2.285361
ENSG00000283695       0.000000
ENSG00000283696      36.321853
ENSG00000283698       8.254363
ENSG00000283699       0.000000
Length: 55169, dtype: float64

In [12]:
df_step = new_df2.sub(min_val, axis=0)
df_normalize = df_step.div(max_min_diff, axis=0)
df_normalize

,205_2,251_1,474_3,178_4,313_1,484_4,288_1,106_4,215_2,184_3,...,330_2,11_4,480_1,314_1,371_3,443_2,311_1,225_2,150_4,289_2
ENSG00000000419,0.225304,0.335718,0.297448,0.284298,0.347123,0.345873,0.357551,0.243153,0.470473,0.171075,...,0.186441,0.570327,0.393986,0.519895,0.265649,0.469552,0.322077,0.330094,0.545498,0.292179
ENSG00000000457,0.313578,0.512403,0.484755,0.405377,0.452662,0.365289,0.506467,0.295023,0.329096,0.397094,...,0.284433,0.480778,0.568303,0.506111,0.570621,0.337235,0.611289,0.583297,0.485943,0.844778
ENSG00000000460,0.217627,0.200999,0.292386,0.218400,0.645533,0.153649,0.239910,0.475373,0.343657,0.208420,...,0.316697,0.294917,0.264831,0.340395,0.260372,0.480329,0.557413,0.103690,0.558471,0.547625
ENSG00000000938,0.308731,0.178393,0.165441,0.136211,0.177183,0.177665,0.170851,0.232388,0.139025,0.215309,...,0.176478,0.258806,0.611819,1.000000,0.131200,0.091485,0.204828,0.180066,0.319826,0.024781
ENSG00000000971,0.568183,0.096086,0.084771,0.182065,0.216948,0.174867,0.180673,0.319593,0.357724,0.045820,...,0.092109,0.238078,0.287420,0.316267,0.179841,0.453220,0.174134,0.070354,0.547662,0.162202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283693,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000283695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ENSG00000283696,0.420977,0.083209,0.083071,0.346737,0.088585,0.088811,0.000000,0.213741,0.296807,0.223619,...,0.185332,0.000000,0.190566,0.312883,0.072148,0.169218,0.418206,0.000000,0.094687,0.476808
ENSG00000283698,0.370486,0.366147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.327999,...,0.000000,0.000000,0.419275,0.000000,0.000000,0.000000,0.000000,0.395756,0.000000,0.000000


In [13]:
# take the mean across the patients
low_var = df_normalize[df_normalize.mean(axis=1)>0.194]
low_var

,205_2,251_1,474_3,178_4,313_1,484_4,288_1,106_4,215_2,184_3,...,330_2,11_4,480_1,314_1,371_3,443_2,311_1,225_2,150_4,289_2
ENSG00000000419,0.225304,0.335718,0.297448,0.284298,0.347123,0.345873,0.357551,0.243153,0.470473,0.171075,...,0.186441,0.570327,0.393986,0.519895,0.265649,0.469552,0.322077,0.330094,0.545498,0.292179
ENSG00000000457,0.313578,0.512403,0.484755,0.405377,0.452662,0.365289,0.506467,0.295023,0.329096,0.397094,...,0.284433,0.480778,0.568303,0.506111,0.570621,0.337235,0.611289,0.583297,0.485943,0.844778
ENSG00000000460,0.217627,0.200999,0.292386,0.218400,0.645533,0.153649,0.239910,0.475373,0.343657,0.208420,...,0.316697,0.294917,0.264831,0.340395,0.260372,0.480329,0.557413,0.103690,0.558471,0.547625
ENSG00000000938,0.308731,0.178393,0.165441,0.136211,0.177183,0.177665,0.170851,0.232388,0.139025,0.215309,...,0.176478,0.258806,0.611819,1.000000,0.131200,0.091485,0.204828,0.180066,0.319826,0.024781
ENSG00000000971,0.568183,0.096086,0.084771,0.182065,0.216948,0.174867,0.180673,0.319593,0.357724,0.045820,...,0.092109,0.238078,0.287420,0.316267,0.179841,0.453220,0.174134,0.070354,0.547662,0.162202
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283623,0.242505,0.384265,0.382538,0.479618,0.309628,0.116237,0.259439,0.010292,0.063012,0.359076,...,0.241619,0.000000,0.320945,0.086969,0.334447,0.030555,0.292350,0.453725,0.000000,1.000000
ENSG00000283633,0.805910,0.118728,0.000000,0.098950,0.271546,0.105601,0.225175,0.238139,0.047056,0.208285,...,0.146914,0.023473,0.627664,0.335205,0.120103,0.281693,0.119345,0.149718,0.315248,0.164699
ENSG00000283667,0.372024,0.275750,0.330352,0.275776,0.626269,0.333555,0.344237,0.306940,0.349725,0.214084,...,0.232023,0.588790,0.336812,0.414750,0.191275,0.168233,0.110873,0.317919,0.774010,0.421363
ENSG00000283674,0.175686,0.171646,0.160965,0.427998,0.160475,0.353705,0.374211,0.062286,0.135035,0.287363,...,0.176976,0.256949,0.127865,0.393364,0.171896,0.141877,0.225985,0.219578,0.087569,0.076299


In [14]:
df = pd.read_csv('batchcorrected_log2cpm.tsv', sep='\t',index_col=0)

In [15]:
df

,100_2,101_3,102_2,103_3,104_2,105_2,106_4,107_4,109_1,11_4,...,90_2,91_2,92_3,93_2,94_4,95_4,96_3,97_2,98_3,99_1
ENSG00000000003,4.224093,4.088110,4.108960,4.338494,4.144095,3.961678,4.068081,4.181222,4.177308,4.345819,...,4.095915,4.185599,3.984862,4.437899,4.258712,3.980472,4.310913,4.078538,4.171295,4.225876
ENSG00000000419,4.273573,4.278318,4.508365,4.533990,4.451600,4.437512,4.229555,4.359195,4.299986,4.569697,...,4.615619,4.413496,4.623100,4.781393,4.534774,4.570218,4.722600,4.492435,4.560072,4.415542
ENSG00000000457,4.716466,4.969749,4.852762,4.947713,4.882771,4.704028,4.675137,4.517319,4.740297,4.672500,...,4.700506,4.839435,4.763086,4.870351,4.876640,4.812464,4.812821,4.840323,4.880159,4.652706
ENSG00000000460,4.126170,4.528228,4.286024,4.310150,4.150902,4.265058,4.737984,4.102466,4.593532,4.613472,...,4.476497,4.568329,4.556887,4.323269,4.165587,4.667403,4.378113,4.334682,4.361387,3.685774
ENSG00000000938,1.171196,0.862429,1.540143,1.105556,1.375205,1.568452,1.638125,1.233508,1.497463,1.548793,...,1.454494,0.874062,1.199031,2.341509,1.732493,1.523060,1.136348,1.578141,0.897523,1.096682
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000283623,0.797770,1.070189,0.841682,0.387205,1.100907,0.563784,0.629173,0.468014,0.999594,0.603904,...,1.024131,0.813795,1.387993,1.177119,1.543931,0.990309,0.858025,0.669921,0.503958,0.852608
ENSG00000283633,0.419013,1.483662,0.010696,0.712213,1.155770,0.360396,1.132086,0.133303,0.705493,0.256704,...,0.765210,0.945644,0.689570,1.976703,0.511556,1.274101,1.394655,1.098044,0.842557,0.746310
ENSG00000283662,0.843287,0.637100,0.723624,0.256273,0.919095,0.731227,0.978961,0.912789,0.748250,0.331183,...,0.822570,0.796490,0.507236,1.053061,0.440690,0.536374,1.082211,0.361556,0.712051,0.298914
ENSG00000283667,1.047956,1.125788,1.194329,1.322104,1.381362,1.410026,1.418967,1.347204,1.282221,1.338360,...,1.448628,1.478424,1.683557,1.257088,1.767100,1.105709,1.679140,1.132673,1.176257,1.323125


In [16]:
#low_var_2 = df_normalize[df_normalize.var(axis=1)>0.02363]
#low_var_ind2 = low_var_2.index

In [17]:
## check if they are similar
#low_var_ind = low_var.index
#limma_ind = df.index

In [18]:
"""
count = 0 
for i in low_var_ind2:
    if i in limma_ind:
        count += 1
count
"""

'\ncount = 0 \nfor i in low_var_ind2:\n    if i in limma_ind:\n        count += 1\ncount\n'